In [88]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
import time
import random

import pandas as pd

In [32]:
option = webdriver.ChromeOptions()
option.add_argument("-incognito")
browser = webdriver.Chrome(executable_path="./chromedriver", chrome_options=option)
browser.get("https://publicholidays.us/school-holidays/")
time.sleep(random.uniform(0, 1)+1)

state_list = [i.get_attribute('href') for i in browser.find_elements_by_xpath('//td[@class="column-1"]/a')] + \
[i.get_attribute('href') for i in browser.find_elements_by_xpath('//td[@class="column-2"]/a')]

In [85]:
def get_school_url_list(url):

    option = webdriver.ChromeOptions()
    option.add_argument("-incognito")

    browser = webdriver.Chrome(executable_path="./chromedriver", chrome_options=option)

    browser.get(url)

    time.sleep(random.uniform(0, 1)+3)
    
    
    
    school_list = [i.get_attribute('href') for i in browser.find_elements_by_xpath('//td[@class="column-1"]/a')] + \
[i.get_attribute('href') for i in browser.find_elements_by_xpath('//td[@class="column-2"]/a')]

    browser.close()
    
    return school_list


In [94]:
def school(url):
    option = webdriver.ChromeOptions()
    option.add_argument("-incognito")

    browser = webdriver.Chrome(executable_path="./chromedriver", chrome_options=option)

    browser.get(url)

    time.sleep(random.uniform(0, 1)+3)

    school = url.split("/")[-2]
    state = url.split("/")[-3]
    holiday_date = [i.text for i in  browser.find_elements_by_xpath('//tr//td')]
    holiday_name = [i if i else None for i in holiday_date[::3]]
    start_date = [i if i else None for i in holiday_date[1::3]]
    end_date = [i if i else None for i in holiday_date[2::3]]
    result = []
    n_holiday = len(holiday_name)
    
    state = [state] * n_holiday
    school = [school] * n_holiday    
    item = {"state": state,
            "school": school,
            "holiday_name": holiday_name[i],
            "start_date": start_date[i],
            "end_date": end_date[i]}
    
    result = pd.DataFrame(item)
    
    browser.close()
    
    return result

In [87]:
school_whole_list = []
for url in state_list:
    school_whole_list.append(get_school_url_list(url))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [91]:
from functools import reduce
school_list = reduce(lambda x,y:x+y, school_whole_list)

In [101]:
with open("school_list","w") as f:
    f.write("\n".join(school_list))

In [ ]:
result = []
fail_list = []
with open('result.csv', 'a') as f:
    for index, url in enumerate(school_list):
        if len(result)==0: 
            result= school(url)
            result.to_csv(f, header=False)
            continue
        try:
            df=school(url)
            result=result.append(df)
            df.to_csv(f, header=False)
        except:
            fail_list.append(index)
        if index % 100 ==0: print(index)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """
Exception ignored in: <function Service.__del__ at 0x00000247F2E80A60>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 176, in __del__
    self.stop()
  File "C:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 151, in stop
    self.send_remote_shutdown_command()
  File "C:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 127, in send_remote_shutdown_command
    url_request.urlopen("%s/shutdown" % self.service_url)
  File "C:\ProgramData\Anaconda3\lib\urllib\request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "C:\ProgramData\Anaconda3\lib\urllib\request.py", line 525, in open
    response = self._open(req, data)
  File "C:\ProgramData\Anaconda3\lib\urllib\request.py", line 54